In [2]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import numpy as np
import os
import argparse
import utils
from fer import FER2013
from torch.autograd import Variable
from models import *
from torchvision import models
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import csv

In [3]:
model = models.squeezenet1_1(pretrained=True)
model.classifier[1] = nn.Conv2d(512, 7, kernel_size=(1,1), stride=(1,1)) 

F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
class Opt:
    def __init__(self):
        self.model = 'SqueezeNet'  
        self.dataset = 'FER2013'
        self.bs = 64  
        self.lr = 0.001  
        self.resume = True 

opt = Opt()

In [19]:
use_cuda = torch.cuda.is_available()
best_Testing_acc = 0 
best_Testing_acc_epoch = 0
best_Validation_acc = 0 
best_Validation_acc_epoch = 0
start_epoch = 0  

learning_rate_decay_start = 50  
learning_rate_decay_every = 10
learning_rate_decay_rate = 0.5  

cut_size = 44
total_epoch = 100

path = os.path.join(opt.dataset + '_' + opt.model)

# Lists to save metrics
train_acc_list = []
train_loss_list = []
testing_acc_list = []
testing_loss_list = []
validation_acc_list = []
validation_loss_list = []

In [20]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(cut_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.TenCrop(cut_size),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])

trainset = FER2013(split='Training', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=opt.bs, shuffle=True, num_workers=0)
Testingset = FER2013(split='Testing', transform=transform_test)
Testingloader = torch.utils.data.DataLoader(Testingset, batch_size=opt.bs, shuffle=False, num_workers=0)
Validationset = FER2013(split='Validation', transform=transform_test)
Validationloader = torch.utils.data.DataLoader(Validationset, batch_size=opt.bs, shuffle=False, num_workers=0)

==> Preparing data..


In [21]:
print('==> Building model..')
net = models.squeezenet1_1(pretrained=True)  
net.classifier[1] = nn.Conv2d(512, 7, kernel_size=(1, 1), stride=(1, 1))
net.num_classes = 7

# Initialize weights of the new layer with better values
torch.nn.init.kaiming_normal_(net.classifier[1].weight)
net.classifier[1].bias.data.zero_()

if not os.path.isdir(path):
    print("No checkpoint directory found. Starting training from scratch.")
    opt.resume = False

if opt.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    checkpoint = torch.load(os.path.join(path, 'Validation_model.t7'), map_location=torch.device('cpu'))
    net.load_state_dict(checkpoint['net'])
    best_Testing_acc = checkpoint['best_Testing_acc']
    best_Validation_acc = checkpoint['best_Validation_acc']
    best_Testing_acc_epoch = checkpoint['best_Testing_acc_epoch']
    best_Validation_acc_epoch = checkpoint['best_Validation_acc_epoch']
    start_epoch = checkpoint['best_Validation_acc_epoch'] + 1
else:
    print('==> Starting training from scratch.')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=opt.lr, weight_decay=1e-4)  

==> Building model..
No checkpoint directory found. Starting training from scratch.
==> Starting training from scratch.


In [22]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    global Train_acc
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    if epoch > learning_rate_decay_start and learning_rate_decay_start >= 0:
        frac = (epoch - learning_rate_decay_start) // learning_rate_decay_every
        decay_factor = learning_rate_decay_rate ** frac
        current_lr = opt.lr * decay_factor
        utils.set_lr(optimizer, current_lr)  # set the decayed rate
    else:
        current_lr = opt.lr
    print('learning_rate: %s' % str(current_lr))

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs, targets
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()  
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).sum().item()

        utils.progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                           % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    Train_acc = 100.*correct/total
    train_acc_list.append(Train_acc)
    train_loss_list.append(train_loss / len(trainloader))

In [23]:
def Testing(epoch):
    global Testing_acc
    global best_Testing_acc
    global best_Testing_acc_epoch
    net.eval()
    Testing_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient computation for testing
        for batch_idx, (inputs, targets) in enumerate(Testingloader):
            bs, ncrops, c, h, w = np.shape(inputs)
            inputs = inputs.view(-1, c, h, w)
            inputs, targets = inputs, targets

            outputs = net(inputs)
            outputs_avg = outputs.view(bs, ncrops, -1).mean(1)  # avg over crops
            loss = criterion(outputs_avg, targets)
            Testing_loss += loss.item()
            _, predicted = torch.max(outputs_avg.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).sum().item()

            utils.progress_bar(batch_idx, len(Testingloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                               % (Testing_loss / (batch_idx + 1), 100. * correct / total, correct, total))

    Testing_acc = 100.*correct/total
    if Testing_acc > best_Testing_acc:
        print('Saving..')
        print("best_Testing_acc: %0.3f" % Testing_acc)
        state = {
            'net': net.state_dict(),
            'acc': Testing_acc,
            'epoch': epoch,
        }
        if not os.path.isdir(path):
            os.mkdir(path)
        torch.save(state, os.path.join(path, 'Testing_model.t7'))
        best_Testing_acc = Testing_acc
        best_Testing_acc_epoch = epoch

    testing_acc_list.append(Testing_acc)
    testing_loss_list.append(Testing_loss / len(Testingloader))

In [24]:
def Validation(epoch):
    global Validation_acc
    global best_Validation_acc
    global best_Validation_acc_epoch
    net.eval()
    Validation_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for batch_idx, (inputs, targets) in enumerate(Validationloader):
            bs, ncrops, c, h, w = np.shape(inputs)
            inputs = inputs.view(-1, c, h, w)
            inputs, targets = inputs, targets

            outputs = net(inputs)
            outputs_avg = outputs.view(bs, ncrops, -1).mean(1)  # avg over crops
            loss = criterion(outputs_avg, targets)
            Validation_loss += loss.item()

            _, predicted = torch.max(outputs_avg.data, 1)
            correct += predicted.eq(targets.data).sum().item()
            total += targets.size(0)  # Update the total number of samples

            utils.progress_bar(batch_idx, len(Validationloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                               % (Validation_loss / (batch_idx + 1), 100. * correct / total, correct, total))

    Validation_acc = 100. * correct / total
    if Validation_acc > best_Validation_acc:
        print('Saving..')
        print("best_Validation_acc: %0.3f" % Validation_acc)
        state = {
            'net': net.state_dict(),
            'acc': Validation_acc,
            'epoch': epoch,
        }
        if not os.path.isdir(path):
            os.mkdir(path)
        torch.save(state, os.path.join(path, 'Validation_model.t7'))
        best_Validation_acc = Validation_acc
        best_Validation_acc_epoch = epoch

    validation_acc_list.append(Validation_acc)
    validation_loss_list.append(Validation_loss / len(Validationloader))

In [25]:
for epoch in range(start_epoch, total_epoch):
    train(epoch)
    Testing(epoch)
    Validation(epoch)


Epoch: 0
learning_rate: 0.001
 [=============================>] | Loss: 1.837 | Acc: 27.821% (7987/28709)     449/449 ===================>..........] | Loss: 1.923 | Acc: 25.190% (4772/18944)     296/449 ======================>.......] | Loss: 1.898 | Acc: 25.758% (5572/21632)     338/449 ========================>.....] | Loss: 1.880 | Acc: 26.326% (6133/23296)     364/449 
 [=============================>] | Loss: 1.630 | Acc: 34.884% (1252/3589)      57/57 
Saving..
best_Testing_acc: 34.884
 [=============================>] | Loss: 1.630 | Acc: 34.884% (1252/3589)      57/57 
Saving..
best_Validation_acc: 34.884

Epoch: 1
learning_rate: 0.001
 [=============================>] | Loss: 1.537 | Acc: 40.088% (11509/28709)    449/449 
 [=============================>] | Loss: 1.390 | Acc: 46.447% (1667/3589)      57/57 
Saving..
best_Testing_acc: 46.447
 [=============================>] | Loss: 1.390 | Acc: 46.447% (1667/3589)      57/57 
Saving..
best_Validation_acc: 46.447

Epoch: 2
le

In [26]:
print("Best Testing accuracy: %0.3f%% at epoch %d" % (best_Testing_acc, best_Testing_acc_epoch))
print("Best Validation accuracy: %0.3f%% at epoch %d" % (best_Validation_acc, best_Validation_acc_epoch))

Best Testing accuracy: 67.373% at epoch 96
Best Validation accuracy: 67.373% at epoch 96
